In [1]:
import pandas as pd
import os
import random

In [ ]:
class MovieDataLoader:
    def __init__(self, file_path = '../data'):
        self.file_path = file_path
        self.__load_data()

    def __load_data(self):
        for file_name in os.listdir(self.file_path):
            self.file_name = []
            if file_name.endswith('.csv'):
                setattr(self, os.path.splitext(file_name)[0], pd.read_csv(os.path.join(self.file_path, file_name)))
            if file_name.endswith('.txt'):
                setattr(self, os.path.splitext(file_name)[0], pd.read_csv(os.path.join(self.file_path, file_name), sep='\t', engine='python'))


class RandomRecommender:
    def __init__(self, movie_data_loader):
        self.loader = movie_data_loader
        self.movies = self.loader.movies
        self.rates = self.loader.rates

    def run(self):
        self.rates['rate_random'] = [random.randrange(1, 11) for _ in range(len(self.rates))]
        self.rates = self.rates.sort_values(by='rate_random', ascending=False)
        return self.rates

movie_data_loader = MovieDataLoader("../data")
data = RandomRecommender(movie_data_loader).run()
data.head(10)

,user,movie,rate,time,rate_random
12,0,10047,8,1453603020,10
140709,52027,10998,10,1227036960,10
140687,52005,10998,9,1472986980,10
140685,52003,10998,10,1375379940,10
13,0,10048,7,1394114760,10
42061,3899,10002,10,1230176940,10
42062,3899,10501,10,1275747060,10
42091,3900,10253,1,1222924080,10
42100,3902,10114,10,1187627400,10
42106,3903,10003,10,1410113700,10


## 평균 절대 오차 (MAE), 평균 제곱 오차 (MSE), 평균 제곱근 오차 (RMSE), 평균 절대 비율 오차 (MAPE) 설명

### **1. 평균 절대 오차 (MAE: Mean Absolute Error)**
- **정의**: 예측값과 실제값 간의 차이를 절댓값으로 변환한 뒤, 그 평균을 계산한 값.
- **공식**:
  $$
  MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
  $$
  여기서 $y_i$는 실제값, $\hat{y}_i$는 예측값, $n$은 데이터 개수.
- **특징**:
  - 데이터의 단위와 동일하여 해석이 직관적.
  - 이상치(outlier)에 덜 민감.
- **단점**:
  - 예측값이 실제값보다 높거나 낮은지에 대한 정보 제공 부족.

---

### **2. 평균 제곱 오차 (MSE: Mean Squared Error)**
- **정의**: 예측값과 실제값 간의 차이를 제곱한 뒤, 그 평균을 계산한 값.
- **공식**:
  $$
  MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
  $$
- **특징**:
  - 큰 오차(이상치)에 더 민감하게 반응.
- **장점**:
  - 계산이 간단하며, 모델 성능 평가에 널리 사용됨.
- **단점**:
  - 제곱으로 인해 에러 크기가 과장될 수 있으며, 단위가 원래 데이터와 다름.

---

### **3. 평균 제곱근 오차 (RMSE: Root Mean Squared Error)**
- **정의**: MSE 값에 제곱근을 씌운 값으로, 데이터 단위를 원래 값과 동일하게 맞춤.
- **공식**:
  $$
  RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
  $$
- **특징**:
  - MSE보다 직관적이며, 실제값과 동일한 단위를 가짐.
- **장점**:
  - 이상치에 민감하지만, MSE보다 왜곡이 적음.
- **단점**:
  - 여전히 이상치에 영향을 받으며, 스케일 의존적.

---

### **4. 평균 절대 비율 오차 (MAPE: Mean Absolute Percentage Error)**
- **정의**: 예측값과 실제값 간의 상대적인 차이를 백분율로 표현한 값.
- **공식**:
  $$
  MAPE = \frac{100}{n} \sum_{i=1}^{n} \left| \frac{y_i - \hat{y}_i}{y_i} \right|
  $$
- **특징**:
  - 모델 성능을 퍼센트로 표현하여 비교가 용이함.
- **장점**:
  - 스케일에 독립적이며 다른 모델 간 비교가 쉬움.
- **단점**:
  - 실제값이 작은 경우 값이 크게 왜곡될 수 있음.
  - 실제값이 $0$인 경우 계산 불가.

---

### 비교 요약
| 지표   | 공식                                                                 | 특징                              | 장점                                | 단점                                  |
|--------|----------------------------------------------------------------------|-----------------------------------|-------------------------------------|---------------------------------------|
| MAE    | $\frac{1}{n} \sum_{i=1}^{n} \|y_i - \hat{y}_i\|$           | 절대 오차의 평균                  | 직관적이고 이상치에 덜 민감          | 방향성 정보 부족                     |
| MSE    | $\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$    | 제곱 오차의 평균                  | 계산 용이, 널리 사용됨              | 이상치에 민감, 단위 불일치            |
| RMSE   | $\sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$   | MSE의 제곱근                     | 직관적이고 단위 일치                | 이상치에 민감                        |
| MAPE   | $\frac{100}{n} \sum_{i=1}^{n} \left\| \frac{y_i - \hat{y}_i}{y_i} \right\|$ | 상대적 오차를 백분율로 표현       | 스케일 독립적, 비교 용이            | 작은 값에서 왜곡 가능                |

